In [31]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical

from Bio import SeqIO
from Bio.Data import IUPACData 
import csv
import numpy as np
import tensorflow as tf

#data_path = 'features_CENH3_DMR6_LUCA-CHLRE00002_orthologues.csv'
#data_path = 'features_oma-seqs-viridiplantae_test-7-8.csv'
data_path = 'features_oma-seqs-viridiplantae_test-5-6-7-8.csv'

In [32]:
def protein2integer(in_seq):
    
    ## define universe of possible input values
    all_protein_letters = list(IUPACData.extended_protein_letters)
    #print(all_protein_letters)
    ## define a mapping of chars to integers 
    ## i+1 beacuse we want to start from integer 1 instead of 0. 0 will be used for padding
    char_to_int = dict((c, i+1) for i, c in enumerate(all_protein_letters))
    int_to_char = dict((i+1, c) for i, c in enumerate(all_protein_letters))
    ## integer encode input data
    integer_encoded = [char_to_int[char] for char in in_seq.upper()]
    
    
    #return(integer_encoded,len(all_protein_letters))
    return(integer_encoded)
    

In [33]:
def make_dataset(in_file):
    with open(in_file, 'r') as f:
        reader = csv.reader(f, delimiter="\t")
        # get all the rows as a list
        d_set = list(reader)
        # transform data into numpy array
        d_set = np.array(d_set).astype(str)
        
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    
    G = d_set[:, 0]
    X = integer_encoded_proteins
    Y = d_set[:, 2].astype(int)
                         
    return(d_set,G,X,Y)

In [34]:
def make_train_test_set(G,X,Y):
    indices = np.random.permutation(X.shape[0])
    train_size = int(indices.size*0.80)
    train_idx, test_idx = indices[:train_size], indices[train_size:]
    #print(len(train_idx),len(test_idx))
    
    X_train, X_test = X[train_idx,], X[test_idx,]
    #print(X_train.shape,X_test.shape)
    
    y_train, y_test = Y[train_idx,], Y[test_idx,]
    
    #print(X[train_idx[0],])
    #print(Y[train_idx[0],])

    #print(X_train[0,])
    #print(y_train[0,])
    
    return(X_train,y_train,X_test,y_test)

In [35]:
def model1(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
    # create the model
    embedding_vecor_length = 4
    model = Sequential()
    model.add(Embedding(num_letters, embedding_vecor_length, input_length=fixed_seq_length))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(75))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())
    
#''' 
#    model.fit(X_train_new, y_train, epochs=in_epochs, batch_size=in_batch_size) #epochs=3, batch_size=64)
#    ## Final evaluation of the model
#    scores = model.evaluate(X_test_new, y_test, verbose=0)
#    print("Accuracy: %.2f%%" % (scores[1]*100))
#'''


    # Convert labels to categorical one-hot encoding & fit the model
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

#    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
#    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, verbose=1)

    # evaluate the model
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()


In [36]:
def model2(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # Initializing the Sequential model from KERAS.
    model = Sequential()

    # Creating a 16 neuron hidden layer with Linear Rectified activation function.
    #model.add(Dense(16, input_dim=1, init='uniform', activation='relu'))
    model.add(Dense(16, input_dim=fixed_seq_length, kernel_initializer='uniform', activation='relu'))

    # Creating a 8 neuron hidden layer.
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))

    # Adding a output layer.
    model.add(Dense(n_classes, kernel_initializer='uniform', activation='softmax'))
    
    # Compiling the model
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

    print(model.summary())
   
    # Convert labels to categorical one-hot encoding & fit the model
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

    # fit the model
#    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
#    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, verbose=1)

    # evaluate the model
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()

In [37]:
def one_hot_matrix(labels,C):
    
    C = tf.constant(C,name="C")
    one_hot_matrix = tf.one_hot(labels,C,axis=1)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [38]:
def model3(X_train_new, y_train,X_test_new, y_test, batch_size =100, hm_epochs =100): # CNN: Convolutional Neural Networks
    # Number of nodes in each NN hidden layer
    n_nodes_hl1 = 1500
    n_nodes_hl2 = 1500
    n_nodes_hl3 = 1500

    # Number of orthology clusters
    #n_classes = len(np.unique(np.concatenate((y_train,y_test),axis=0)))     #2 or 3 or ...
    
    #y_all = np.concatenate((y_train,y_test),axis=0)
    #y_min = np.amin(y_all)
    #n_classes = np.amax(y_all-y_min)+1
    
    
    train_y = one_hot_matrix(y_train,n_classes)
    test_y = one_hot_matrix(y_test,n_classes)

    # Batch size and Epoch size for training the NN
    #batch_size = 100   #100
    #hm_epochs = 100    #1000

    # Initializing X and Y
    x = tf.placeholder('float')
    y = tf.placeholder('float')

    # Initializing NN layers
    hidden_1_layer = {'f_fum':n_nodes_hl1,
                  'weight':tf.Variable(tf.random_normal([len(X_train_new[0]), n_nodes_hl1])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'f_fum':n_nodes_hl2,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'f_fum':n_nodes_hl3,
                  'weight':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                  'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'f_fum':None,
                'weight':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                'bias':tf.Variable(tf.random_normal([n_classes])),}


    
    
    l1 = tf.add(tf.matmul(x,hidden_1_layer['weight']), hidden_1_layer['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weight']), hidden_2_layer['bias'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weight']), hidden_3_layer['bias'])
    l3 = tf.nn.relu(l3)

    prediction = tf.matmul(l3,output_layer['weight']) + output_layer['bias']

        
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #try:
        #    epoch = int(open(tf_log,'r').read().split('\n')[-2])+1
        #    print('STARTING:',epoch)
        #except:
        #    epoch = 1
        epoch = 1

        while epoch <= hm_epochs:
            epoch_loss = 1
            
            i=0
            while i < len(X_train_new):
                start = i
                end = i+batch_size
                batch_x = np.array(X_train_new[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,y: batch_y})
                epoch_loss += c
                i+=batch_size
                
            
            print('Epoch ',epoch,' out of ',hm_epochs,'- loss:',epoch_loss)
 
            
            #with open(tf_log,'a') as f:
            #    f.write(str(epoch)+'\n') 
            epoch +=1
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        #print("\nModel saved in path: %s " % my_model_save_path)
        print('\nAccuracy:',accuracy.eval({x:X_test_new, y:test_y}) * 100)
    return()

In [39]:
def model4(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    
    # Convert labels to categorical one-hot encoding  
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    X_train_new_one_hot_labels = np.array([to_categorical(x, num_classes=num_letters) for x in X_train_new])
    X_test_new_one_hot_labels = np.array([to_categorical(x, num_classes=num_letters) for x in X_test_new])

    # create the model    
    model = Sequential()
    model.add(LSTM(75,input_shape=X_train_new_one_hot_labels[0].shape,return_sequences=True))
    model.add(LSTM(75))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())  

    # fit the model
    model.fit(X_train_new_one_hot_labels, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size, verbose=1)

    # evaluate the model
    loss, accuracy = model.evaluate(X_test_new_one_hot_labels, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()


In [40]:
def model5(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # LSTM
    
    # Convert labels to categorical one-hot encoding  
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)

    # create the model    
    model = Sequential()
    model.add(Embedding(num_letters, output_dim=fixed_seq_length))
    model.add(LSTM(75,return_sequences=True))
    model.add(LSTM(75))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())

    # fit the model
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

    # evaluate the model
    loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()


In [41]:
def model6(X_train_new, y_train,X_test_new, y_test,in_batch_size=100,in_epochs=10): # RNN: Recurrent Neural Networks
    # GRU
    
    # Convert labels to categorical one-hot encoding  
    y_train_one_hot_labels = to_categorical(y_train, num_classes=n_classes)
    y_test_one_hot_labels = to_categorical(y_test, num_classes=n_classes)
    
    # create the model    
    model = Sequential()
    model.add(Embedding(num_letters, output_dim=fixed_seq_length))
    model.add(GRU(128, return_sequences=False))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())

    # fit the model
    model.fit(X_train_new, y_train_one_hot_labels, epochs=in_epochs, batch_size=in_batch_size)

    # evaluate the model
    loss, accuracy = model.evaluate(X_test_new, y_test_one_hot_labels, verbose=0)
    print('Accuracy: %f' % (accuracy*100))
    
    return()


In [42]:
def test_1(d_set):
    print(d_set[:,0])
    print(d_set[0:2,1])
    print(d_set[:,2].astype(int))
    print(d_set.shape)
    print(d_set[:,1].shape)
    return()

In [43]:
def test_2(d_set):
    integer_encoded_proteins = np.array([protein2integer(seq) for seq in d_set[:,1]])
    print(len(integer_encoded_proteins))
    print(integer_encoded_proteins[0])
    #np.array(integer_encoded_proteins).shape
    print(integer_encoded_proteins.shape)
    #protein2integer(dataset[:,1])
    return()

In [44]:
def test_3(G,X,Y):
    print(G.shape)
    print(X.shape)
    print(Y.shape)

    print(G[0:3,])
    print(X[0:3,])
    print(Y[0:3,])
    
    print('Maximum review length: {}'.format(len(max(X, key=len))))
    print('Minimum review length: {}'.format(len(min(X, key=len))))
    
    return()

In [45]:
def test_4(X_train_new,X_train):
    print(X_train_new.shape)
    print(X_train_new[0,:])
    print(X_train.shape)
    print(X_train[0,])
    return()


In [46]:
dataset, G, X, Y = make_dataset(data_path)
X_train,y_train,X_test,y_test = make_train_test_set(G,X,Y)

#print("============ Test 1 =======================")
#test_1(dataset)
#print("============ Test 2 =======================")
#test_2(dataset)
#print("============ Test 3 =======================")
#test_3(G,X,Y)



In [47]:
num_letters = len(list(IUPACData.extended_protein_letters)) # = 26
#fixed_seq_length = len(max(X, key=len)) # maximum
#fixed_seq_length = (sum(len(X[i,]) for i in range(X.shape[0]))/X.shape[0])  # average
fixed_seq_length = 1000
n_classes = int(np.amax(np.concatenate((y_train,y_test),axis=0))+1)
# truncate and pad input sequences
X_train_new = sequence.pad_sequences(X_train, maxlen=fixed_seq_length, padding='post', truncating='post')
X_test_new = sequence.pad_sequences(X_test, maxlen=fixed_seq_length, padding='post', truncating='post')
  
#print("============ Test 4 =======================")
#test_4(X_train_new,X_train)


In [35]:
model1(X_train_new, y_train, X_test_new, y_test,200,1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 1000, 4)           104       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1000, 32)          416       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 500, 32)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 75)                32400     
_________________________________________________________________
dense_19 (Dense)             (None, 10219)             776644    
Total params: 809,564
Trainable params: 809,564
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
3505/3505 [==============================] - 14s 4ms/step - loss: 8.5610 - acc: 2.8531e-04
Epoch 2/1000
3505/3505 [===

3505/3505 [==============================] - 13s 4ms/step - loss: 4.2522 - acc: 0.0437
Epoch 152/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 4.2290 - acc: 0.0511
Epoch 153/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 4.2383 - acc: 0.0462
Epoch 154/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 4.2267 - acc: 0.0491
Epoch 155/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 4.2284 - acc: 0.0462
Epoch 156/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 4.2180 - acc: 0.0485
Epoch 157/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 4.2317 - acc: 0.0462
Epoch 158/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 4.2203 - acc: 0.0536
Epoch 159/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 4.2068 - acc: 0.0462
Epoch 160/1000
3505/3505 [==============================] - 12s 4ms/step - loss: 4.2128 - acc: 0.

3505/3505 [==============================] - 13s 4ms/step - loss: 3.9155 - acc: 0.0759
Epoch 312/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.9181 - acc: 0.0762
Epoch 313/1000
3505/3505 [==============================] - 12s 4ms/step - loss: 3.9213 - acc: 0.0753
Epoch 314/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.9148 - acc: 0.0719
Epoch 315/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.9156 - acc: 0.0685
Epoch 316/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.9016 - acc: 0.0799
Epoch 317/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.9085 - acc: 0.0816
Epoch 318/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.9100 - acc: 0.0716
Epoch 319/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.9214 - acc: 0.0759
Epoch 320/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.9009 - acc: 0.

3505/3505 [==============================] - 12s 3ms/step - loss: 3.6776 - acc: 0.0944
Epoch 472/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.6999 - acc: 0.0999
Epoch 473/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.7000 - acc: 0.0950
Epoch 474/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.6730 - acc: 0.1027
Epoch 475/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.6984 - acc: 0.0916
Epoch 476/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.6906 - acc: 0.0942
Epoch 477/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.6951 - acc: 0.0961
Epoch 478/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.6623 - acc: 0.1058
Epoch 479/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.6740 - acc: 0.1016
Epoch 480/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.6879 - acc: 0.

3505/3505 [==============================] - 12s 3ms/step - loss: 3.4440 - acc: 0.1275
Epoch 632/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.4744 - acc: 0.1238
Epoch 633/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.4243 - acc: 0.1304
Epoch 634/1000
3505/3505 [==============================] - 12s 4ms/step - loss: 3.4646 - acc: 0.1272
Epoch 635/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.4659 - acc: 0.1244
Epoch 636/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.4232 - acc: 0.1355
Epoch 637/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.4334 - acc: 0.1304
Epoch 638/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.4333 - acc: 0.1312
Epoch 639/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.4224 - acc: 0.1330
Epoch 640/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.4335 - acc: 0.

3505/3505 [==============================] - 13s 4ms/step - loss: 3.1851 - acc: 0.1621
Epoch 792/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.1479 - acc: 0.1735
Epoch 793/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.2412 - acc: 0.1609
Epoch 794/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.1811 - acc: 0.1623
Epoch 795/1000
3505/3505 [==============================] - 12s 4ms/step - loss: 3.1689 - acc: 0.1666
Epoch 796/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.1796 - acc: 0.1603
Epoch 797/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.1762 - acc: 0.1660
Epoch 798/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 3.1602 - acc: 0.1726
Epoch 799/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 3.1619 - acc: 0.1726
Epoch 800/1000
3505/3505 [==============================] - 12s 4ms/step - loss: 3.1551 - acc: 0.

3505/3505 [==============================] - 12s 3ms/step - loss: 2.9472 - acc: 0.2091
Epoch 952/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 2.9293 - acc: 0.2106
Epoch 953/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 2.9625 - acc: 0.2031
Epoch 954/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 2.9640 - acc: 0.2029
Epoch 955/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 2.9011 - acc: 0.2163
Epoch 956/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 2.9814 - acc: 0.2091
Epoch 957/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 2.9031 - acc: 0.2203
Epoch 958/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 2.9130 - acc: 0.2160
Epoch 959/1000
3505/3505 [==============================] - 12s 3ms/step - loss: 2.9339 - acc: 0.2040
Epoch 960/1000
3505/3505 [==============================] - 13s 4ms/step - loss: 2.8835 - acc: 0.

()

In [ ]:
model2(X_train_new, y_train, X_test_new, y_test,200,1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 16)                16016     
_________________________________________________________________
dense_20 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_21 (Dense)             (None, 15999)             143991    
Total params: 160,143
Trainable params: 160,143
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
9324/9324 [==============================] - 5s 585us/step - loss: 9.0726 - acc: 5.3625e-04
Epoch 2/1000
9324/9324 [==============================] - 5s 499us/step - loss: 7.8403 - acc: 7.5075e-04
Epoch 3/1000
9324/9324 [==============================] - 5s 499us/step - loss: 7.4961 - acc: 0.0012
Epoch 4/1000
9324/9324 [==============================] - 5s 500us/step - loss: 7.23

9324/9324 [==============================] - 5s 574us/step - loss: 4.9577 - acc: 0.0526
Epoch 73/1000
9324/9324 [==============================] - 5s 568us/step - loss: 4.9500 - acc: 0.0542
Epoch 74/1000
9324/9324 [==============================] - 5s 574us/step - loss: 4.9446 - acc: 0.0575
Epoch 75/1000
9324/9324 [==============================] - 5s 567us/step - loss: 4.9287 - acc: 0.0557
Epoch 76/1000
9324/9324 [==============================] - 5s 569us/step - loss: 4.9217 - acc: 0.0580
Epoch 77/1000
9324/9324 [==============================] - 5s 570us/step - loss: 4.9126 - acc: 0.0596
Epoch 78/1000
9324/9324 [==============================] - 5s 571us/step - loss: 4.9009 - acc: 0.0587
Epoch 79/1000
9324/9324 [==============================] - 5s 569us/step - loss: 4.8997 - acc: 0.0593
Epoch 80/1000
9324/9324 [==============================] - 5s 564us/step - loss: 4.8853 - acc: 0.0591
Epoch 81/1000
9324/9324 [==============================] - 5s 570us/step - loss: 4.8845 - acc: 0

Epoch 152/1000
9324/9324 [==============================] - 5s 569us/step - loss: 4.3622 - acc: 0.1170
Epoch 153/1000
9324/9324 [==============================] - 5s 569us/step - loss: 4.3722 - acc: 0.1143
Epoch 154/1000
9324/9324 [==============================] - 5s 569us/step - loss: 4.3704 - acc: 0.1164
Epoch 155/1000
9324/9324 [==============================] - 5s 568us/step - loss: 4.3570 - acc: 0.1201
Epoch 156/1000
9324/9324 [==============================] - 5s 566us/step - loss: 4.3431 - acc: 0.1224
Epoch 157/1000
9324/9324 [==============================] - 5s 568us/step - loss: 4.3312 - acc: 0.1213
Epoch 158/1000
9324/9324 [==============================] - 5s 571us/step - loss: 4.3331 - acc: 0.1207
Epoch 159/1000
9324/9324 [==============================] - 5s 571us/step - loss: 4.3333 - acc: 0.1210
Epoch 160/1000
9324/9324 [==============================] - 5s 571us/step - loss: 4.3188 - acc: 0.1195
Epoch 161/1000
9324/9324 [==============================] - 5s 571us/step

9324/9324 [==============================] - 5s 548us/step - loss: 3.9834 - acc: 0.1717
Epoch 232/1000
9324/9324 [==============================] - 5s 531us/step - loss: 3.9996 - acc: 0.1610
Epoch 233/1000
9324/9324 [==============================] - 5s 530us/step - loss: 3.9875 - acc: 0.1654
Epoch 234/1000
9324/9324 [==============================] - 5s 550us/step - loss: 3.9777 - acc: 0.1658
Epoch 235/1000
9324/9324 [==============================] - 5s 578us/step - loss: 3.9766 - acc: 0.1697
Epoch 236/1000
9324/9324 [==============================] - 5s 585us/step - loss: 3.9753 - acc: 0.1691
Epoch 237/1000
9324/9324 [==============================] - 5s 547us/step - loss: 3.9730 - acc: 0.1637
Epoch 238/1000
9324/9324 [==============================] - 5s 539us/step - loss: 3.9672 - acc: 0.1671
Epoch 239/1000
9324/9324 [==============================] - 5s 571us/step - loss: 3.9769 - acc: 0.1668
Epoch 240/1000
9324/9324 [==============================] - 4s 474us/step - loss: 3.9689

model3(X_train_new, y_train, X_test_new, y_test,200,100)

In [25]:
model4(X_train_new, y_train, X_test_new, y_test,200,10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1000, 75)          30600     
_________________________________________________________________
lstm_6 (LSTM)                (None, 75)                45300     
_________________________________________________________________
dense_5 (Dense)              (None, 10219)             776644    
Total params: 852,544
Trainable params: 852,544
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 0.000000


()

In [19]:
model5(X_train_new, y_train, X_test_new, y_test,200,50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 1000)        26000     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 75)          322800    
_________________________________________________________________
lstm_2 (LSTM)                (None, 75)                45300     
_________________________________________________________________
dense_1 (Dense)              (None, 10219)             776644    
Total params: 1,170,744
Trainable params: 1,170,744
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
3505/3505 [==============================] - 155s 44ms/step - loss: 8.3424 - acc: 5.7061e-04
Epoch 2/50
3505/3505 [==============================] - 153s 44ms/step - loss: 6.9927 - acc: 0.0011
Epoch 3/50
3505/3505 [==============================] -

()

In [ ]:
model6(X_train_new, y_train, X_test_new, y_test,200,100)